### **Modelado de colas de espera en un hospital público de Guatemala**

CC3039 - Modelación y Simulación

Stefano Aragoni, Luis Diego Santos, Carol Arevalo

--------------

#### *Introducción*

El sistema de salud pública en Guatemala, como en muchos otros países, enfrenta desafíos significativos. Uno de los más críticos es la prolongada espera que los pacientes deben soportar antes de recibir atención médica. Estas colas de espera pueden tener consecuencias graves, incluida una atención médica de menor calidad y una experiencia del paciente desfavorable. Este proyecto busca abordar este problema a través de técnicas avanzadas de modelado y simulación.

--------------

#### *Hospital Roosevelt de Guatemala*

El Hospital Roosevelt comenzó su construcción en 1944 en lo que antes era la finca "La Esperanza". Sin embargo, no fue sino hasta 1945 que abrió oficialmente sus puertas. Gracias al generoso apoyo internacional, este hospital experimentó un rápido crecimiento. Para agosto de 1945, ya se había planificado la expansión de su capacidad a 1,000 camas y la creación de una Escuela de Enfermería. Este hospital guatemalteco se convirtió en un importante referente de la atención médica en la región. Sin embargo, con el paso del tiempo, comenzó a enfrentar desafíos, como la falta de suministros y largos tiempos de espera, lo que afectó la calidad de la atención médica que podía brindar.


En la actualidad, el Hospital Roosevelt ha enfrentado retos significativos, como los presentados por la pandemia de COVID-19. En junio de 2020, médicos del hospital denunciaron que la capacidad para atender a pacientes con coronavirus había sido sobrepasada, tratando a unos 150 pacientes al día en un espacio diseñado para 100, todos en estado crítico. La saturación del hospital ha llegado a tal punto que se ha tenido que recurrir a la ayuda de especialistas de otras áreas y ha habido momentos en que los pacientes han debido ser atendidos en el suelo por falta de espacio. Además, los médicos expresaron su preocupación por la posible falta futura de insumos médicos, dado que el equipo de protección personal se había calculado para una afluencia menor de pacientes. Esta situación refleja los desafíos que el Hospital Roosevelt y, por extensión, el sistema de salud pública en Guatemala, enfrentan para proporcionar atención oportuna y adecuada a su población.



----------

#### *Estadísticas del Hospital Roosevelt*

Para poder desarrollar este proyecto, fue necesario determinar ciertos aspectos del Hospital Roosevelt, como su capacidad, y el número de pacientes que atiende. Para ello, se realizó una investigación en la que se recopilaron datos de diversas fuentes, como el Ministerio de Salud Pública y Asistencia Social (MSPAS), y el Hospital Roosevelt. A continuación, se presentan los resultados obtenidos.

- Cuenta con tres emergencias maternidad, adultos y pediatria
- Tiene capacidad para 950 pacientes en cama
- Se tine una capcidad de 65 pacientes en el area de emergencias
- Hay 9 camas para observación, 20 bancas para inhaloterapia y 5 cuartos de shock.
- Diariamente atienden a un promedio de 400 pacientes en urgencias.
    - El 20% requieren atención inmediata.
    - El 70% atienden por enfermedades comunes.

Finalmente, a través de una investigación realizada por Prensa Libre se logró obtener el protocolo que se sigue cuando un paciente entra a urgencias. Dicha información fue proveída directamente por doctores y otros colaboradores del Hospital Roosevelt. A continuación, se presentan los puntos más relevantes de dicho protocolo.


1. Médicos evalúan si son casos que requieren atención de urgencia. 
    - Hay 4 clinicas de clasificacion. 
3. El 75% de los casos que entran a urgencias son *enfermedades comunes*.
    - Estos son tratados en la clínica, con un tiempo de espera de 15 minutos.
4. Las *urgencias* son heridas, elevación de presión, deshidratación, o casos que merecen una atención mayor, estos pasan con un especialista.
    - Entre 45 minutos y 3 horas de espera dependiendo de afluencia. 
5. Cuando la persona llega con una herida de arma blanca o de fuego, con traumatismo, politraumatismo o complicaciones de enfermedades crónicas, se trata de una emergencia e ingresan cuanto antes para ser tratada en el área de Traumatología, Medicina interna o Cirugía.
    - Estas personas entran a clinica de *claisificacion 1* directamente (desde ambulancias)
    - 3.3% de los casos son de este tipo.


Referencias:

- Ola, A. (2019). Se reduce la saturación de pacientes en las emergencias de los hospitales. Prensa Libre. https://prensalibre.com/guatemala/comunitario/se-reduce-la-saturacion-de-pacientes-en-las-emergencias-de-los-hospitales/.
- Hospital Roosevelt. (2016). Hospital Roosevelt conmemora 61 años de Aniversario - Hospital Roosevelt. Hospitalroosevelt. https://hospitalroosevelt.gob.gt/hospital-roosevelt-conmemora-61-anos-de-aniversario/.
- Hospital Roosevelt. (2017). Acerca de Hospital Roosevelt. Hospitalroosevelt. https://hospitalroosevelt.com/acerca-de-hospital-roosevelt/.

----------

### *Simulación del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala*

Con los valores anteriormente mencionados, se procedió a realizar la simulación del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala. Esto con el propósito de determinar el tiempo de espera promedio que un paciente debe soportar antes de ser atendido por un médico.

Asimismo, se tiene como objetivo determinar cómo alterar ciertos parámetros del sistema para reducir el tiempo de espera promedio de los pacientes. Para ello, se realizarán simulaciones con diferentes valores de los parámetros y se compararán los resultados obtenidos.

##### *1. Definiciones y Parámetros Iniciales*

Como primer paso, se definieron las siguientes variables y parámetros iniciales:

In [78]:
from mesa import Agent
import random

# Capacidades y parámetros del hospital
CAPACIDAD_EMERGENCIAS = 65
PACIENTES_POR_DIA = 400                              

# Probabilidades para clasificación
PORCENTAJE_URGENCIAS = 0.20                                                 # 20% de probabilidad de urgencias
PROB_EMERGENCIA = 0.033                                                     # 3.3% de probabilidad hospitalización / emergencia
PROB_ENFERMEDADES_COMUNES = 0.7                                             # 70% de probabilidad de enfermedades comunes       


# Tiempo de espera
TIEMPO_ESPERA_COMUNES = 15                                      # 15 minutos para ser atendido por enfermedades comunes
TIEMPO_ESPERA_PROMEDIO = lambda: random.randint(45, 3*60)       # 45 minutos a 3 horas para ser atendido por urgencias


##### *2. Funciones de Simulación*

Posteriormente, se definieron clases y funciones base de la simulación. 

La primera función, **clasificar_paciente, indica el tiempo de espera de un paciente para ser atendido por un doctor**. Esto dependiendo si el paciente es de enfermedad común o urgencia. 

In [79]:
def clasificar_paciente():
    """
    Clasifica al paciente en base a probabilidades y regresa el tiempo de espera correspondiente.
    """
    rand_num = random.random()
    
    if rand_num <= PROB_ENFERMEDADES_COMUNES:
        return TIEMPO_ESPERA_COMUNES
    else:
        return TIEMPO_ESPERA_PROMEDIO

Las primeras clases que se definieron fueron aquellas que representan las diferentes clínicas y áreas de atención del hospital. Estas clases heredan de la clase **Clinica**.

Estas clínicas buscan simular el proceso de clasificación de pacientes que se realiza en el Hospital Roosevelt. Asimismo, se busca simular la asignación del orden de atención de los pacientes, dependiendo de su clasificación. Finalmente, también se logra simular el proceso de atención de los pacientes por parte de los médicos.

In [80]:
class Clinica(Agent):
    def __init__(self, unique_id, model, capacidad=1):
        super().__init__(unique_id, model)
        self.capacidad = capacidad
        self.pacientes_actualmente = 0
        self.tiempo_espera_promedio = 0
        self.total_pacientes_atendidos = 0

    def atender_paciente(self, paciente):
        self.pacientes_actualmente += 1

        # Actualizamos el tiempo de espera promedio
        self.tiempo_espera_promedio = ((self.tiempo_espera_promedio * self.total_pacientes_atendidos) + paciente.tiempo_espera) / (self.total_pacientes_atendidos + 1)
        self.total_pacientes_atendidos += 1

        # Iniciar el tiempo de atención de esta clínica
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

    def termino_atencion(self, paciente):
        self.liberar()
        return True

    def esta_disponible(self):
        return self.pacientes_actualmente < self.capacidad

    def tiempo_atencion(self):
        pass

    def liberar(self):
        self.pacientes_actualmente -= 1
        if self.pacientes_actualmente < 0:
            self.pacientes_actualmente = 0
        self.check_cola_espera()
    
    def check_cola_espera(self):
        if isinstance(self, ClinicaEnfermedadesComunes) and self.model.cola_enfermedad_comun:
            paciente_en_espera = self.model.cola_enfermedad_comun.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'enfermedad_comun'
        elif isinstance(self, ClinicaUrgencias) and self.model.cola_urgencias:
            paciente_en_espera = self.model.cola_urgencias.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'urgencia'
        elif isinstance(self, ClinicaClasificacion1) and self.model.cola_clasificacion_1:
            paciente_en_espera = self.model.cola_clasificacion_1.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'clasificacion_1'
        elif isinstance(self, ClinicaClasificadora) and self.model.cola_clasificadora:
            paciente_en_espera = self.model.cola_clasificadora.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'clasificadora'

class ClinicaEnfermedadesComunes(Clinica):
    def __init__(self, unique_id, model, capacidad=50):    # 50 doctores, 900 camas de enfermedades comunes
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        # Entre 3 y 15 minutos
        return random.randint(5, 15)
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_enfermedad_comun += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion() 
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

class ClinicaUrgencias(Clinica):
    def __init__(self, unique_id, model, capacidad=65):     # 65 doctores de urgencias
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        # Entre 10 y 30 minutos
        return random.randint(25, 30)                       # Entre 10 y 30 minutos de cita con el médico para tratamiento de urgencias
        
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_urgencias += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

class ClinicaClasificacion1(Clinica):
    def __init__(self, unique_id, model, capacidad=10):     # 10 clínicas de clasificación 1
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        return random.randint(45, 120)                      # Entre 45 minutos y 2 horas para operar o tratar al paciente
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_clasificacion_1 += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion() 
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)
    

class ClinicaClasificadora(Clinica):
    def __init__(self, unique_id, model, capacidad=4):      # 4 clínicas clasificadoras
        super().__init__(unique_id, model, capacidad)

    def clasificar_paciente(self):
        r = random.random()
        if r < 0.60:
            return 'enfermedad_comun'
        else:
            return 'urgencia'
        
    def tiempo_atencion(self):
        return random.randint(1, 5)          # Entre 1 y 5 minutos para clasificar al paciente
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_clasificadora += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

    

Posteriormente, se definió la clase **Paciente, la cual contiene los atributos de cada paciente que ingresa al sistema**. Debido a que los pacientes se están tratando como agentes, se crea un objeto nuevo cada vez que un paciente ingresa al sistema.


Esta clase almacena si un paciente está siendo atendido o si está esperando a ser atendido. Asimismo, indica si el paciente entró en ambulancia. Finalmente, almacena el tiempo de espera del paciente.

In [81]:
class Paciente(Agent):
    ESPERANDO = 'entrando'
    SIENDO_ATENDIDO = 'siendo_atendido'
    FINALIZADO = 'finalizado'
    
    def __init__(self, unique_id, model, en_ambulancia=False):
        self.unique_id = unique_id                      # ID único del paciente
        self.model = model            
        self.en_ambulancia = en_ambulancia              # Si el paciente llegó en ambulancia (emergencia)
        self.tiempo_llegada = model.current_time        # Al crear un paciente, registramos el tiempo de llegada
        self.tiempo_espera = 0                          # Inicializamos el tiempo de espera en 0
        self.estado = self.ESPERANDO                    # Inicialmente, el paciente está esperando

    def esperar(self):
        if self.estado == self.ESPERANDO:
            self.tiempo_espera += 1                     # Sumamos 1 minutos a la espera

    def recibir_atencion(self):
        self.estado = self.SIENDO_ATENDIDO              # Cambiamos el estado del paciente a siendo atendido

    def finalizar(self):
        self.estado = self.FINALIZADO                   # Cambiamos el estado del paciente a finalizado 
        

    def step(self):
        self.esperar()
        
        if self.estado == self.ESPERANDO:               # Si el paciente está esperando
            
            if self.en_ambulancia:                      # Si llegó en ambulancia

                clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaClasificacion1)      # Buscamos una clínica clasificadora disponible
                if clinica_target:                                                                      # Si hay una clínica clasificadora disponible
                    self.clinica_asignada = clinica_target                              
                    self.clinica_asignada.atender_paciente(self)                                        # Asignamos la clínica al paciente
                    self.recibir_atencion()                                                             # Cambiamos el estado del paciente a siendo atendido
                    self.model.pacientes_clasificacion_1 += 1                                           # Sumamos 1 al contador de pacientes en clasificación 1

                else:
                    self.model.cola_clasificacion_1.append(self)                                        # Si no hay una clínica clasificadora disponible, agregamos al paciente a la cola de clasificación 1

            else:                                       # Si no llegó en ambulancia
                
                clinica_clasificadora_disponible = self.model.clinica_disponible(tipo_clinica=ClinicaClasificadora)     # Buscamos una clínica clasificadora disponible

                if clinica_clasificadora_disponible:                                                                    # Si hay una clínica clasificadora disponible

                    self.clinica_asignada = clinica_clasificadora_disponible                                            # Asignamos la clínica al paciente
                    self.clinica_asignada.atender_paciente(self)
                    clasificacion = self.clinica_asignada.clasificar_paciente()                                         # Clasificamos al paciente

                    if clasificacion == 'enfermedad_comun':                                                             # Si el paciente es clasificado como enfermedad común
                        clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaEnfermedadesComunes)         # Buscamos una clínica de enfermedades comunes disponible
                    elif clasificacion == 'urgencia':                                                                   # Si el paciente es clasificado como urgencia
                        clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaUrgencias)                   # Buscamos una clínica de urgencias disponible
                    elif clasificacion == 'clasificacion_1':                                                            # Si el paciente es clasificado como clasificación 1
                        clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaClasificacion1)              # Buscamos una clínica de clasificación 1 disponible

                    # Paciente ya fue clasificado, ahora se le asigna una clínica para ser atendido
                    #------------------------------------#
                    self.estado = clinica_target                                                                    # Paciente ahora está esperando ser atendido por la clínica
                    self.tiempo_espera = 0                                                                          # Reiniciamos el tiempo de espera                                                                     
                    #------------------------------------#
                    
                    if clinica_target:                                                                                  # Si hay una clínica del tipo correcto disponible
                        self.clinica_asignada = clinica_target
                        self.clinica_asignada.atender_paciente(self)                                                    # Asignamos la clínica al paciente
                        self.recibir_atencion()                                                                         # Cambiamos el estado del paciente a siendo atendido

                        if clasificacion == 'enfermedad_comun':
                            self.model.pacientes_enfermedad_comun += 1
                        elif clasificacion == 'urgencia':
                            self.model.pacientes_urgencias += 1
                        elif clasificacion == 'clasificacion_1':
                            self.model.pacientes_clasificacion_1 += 1
                        
                    else:                                                                                               # Si no hay una clínica del tipo correcto disponible
                        if clasificacion == 'enfermedad_comun':
                            self.model.cola_enfermedad_comun.append(self)                                               # Agregamos al paciente a la cola correspondiente
                        elif clasificacion == 'urgencia':
                            self.model.cola_urgencias.append(self)
                        elif clasificacion == 'clasificacion_1':
                            self.model.cola_clasificacion_1.append(self)
                
        elif self.estado == self.SIENDO_ATENDIDO:                                               # Si el paciente está siendo atendido
            if self.clinica_asignada and self.clinica_asignada.termino_atencion(self):          # Si la clínica asignada al paciente terminó su atención
                self.finalizar()                                                                # Cambiamos el estado del paciente a finalizado


#### 2. Definición del Modelo:

El modelo controlará la lógica global de la simulación y gestionará todos los agentes.

In [82]:
class Event:
    def __init__(self, tipo, t, clinica):
        self.tipo = tipo
        self.t = t
        self.clinica = clinica


In [83]:
from mesa import Model
from mesa.time import RandomActivation

class HospitalModel(Model):
    def __init__(self, N, num_clinicas_clasificadoras=4, num_clinicas_comunes=1, num_clinicas_urgencias=1, num_clinicas_clasificacion1=1):
        self.num_pacientes = N
        self.upcoming_events = []
        self.schedule = RandomActivation(self)
        self.current_time = 0  # El tiempo actual de la simulación

        # Colas de espera
        self.cola_enfermedad_comun = []
        self.cola_urgencias = []
        self.cola_clasificacion_1 = []
        self.cola_clasificadora = []

        # Iniciar contadores y acumuladores de tiempo de espera
        self.pacientes_enfermedad_comun = 0
        self.pacientes_urgencias = 0
        self.pacientes_clasificacion_1 = 0
        self.tiempo_espera_enfermedad_comun = 0
        self.tiempo_espera_urgencias = 0
        self.tiempo_espera_clasificacion_1 = 0
        self.tiempo_espera_clasificadora = 0

        # Crear clínicas
        self.clinicas_clasificadoras = [ClinicaClasificadora(i, self) for i in range(num_clinicas_clasificadoras)]
        self.clinicas_comunes = [ClinicaEnfermedadesComunes(i, self) for i in range(num_clinicas_comunes)]
        self.clinicas_urgencias = [ClinicaUrgencias(i+len(self.clinicas_comunes), self) for i in range(num_clinicas_urgencias)]
        self.clinicas_clasificacion1 = [ClinicaClasificacion1(i+len(self.clinicas_comunes)+len(self.clinicas_urgencias), self) for i in range(num_clinicas_clasificacion1)]
        self.clinicas = self.clinicas_clasificadoras + self.clinicas_comunes + self.clinicas_urgencias + self.clinicas_clasificacion1

        # Crear pacientes
        for i in range(self.num_pacientes):
            paciente = Paciente(i, self, en_ambulancia=random.random() < 0.033)
            self.schedule.add(paciente)

    def reporte(self):
        print(f"Cantidad de pacientes que entraron: {self.num_pacientes}")
        print(f"Cantidad de pacientes de urgencias: {self.pacientes_urgencias}")
        print(f"Cantidad de pacientes de enfermedad común: {self.pacientes_enfermedad_comun}")
        print(f"Cantidad de pacientes de clasificación 1: {self.pacientes_clasificacion_1}")

        print(f"Promedio de tiempo de espera enfermedad común: {self.tiempo_espera_enfermedad_comun/self.pacientes_enfermedad_comun}")
        print(f"Promedio de tiempo de espera urgencias: {self.tiempo_espera_urgencias/self.pacientes_urgencias}")
        print(f"Promedio de tiempo de espera clasificación 1: {self.tiempo_espera_clasificacion_1/self.pacientes_clasificacion_1}")
        print(f"Promedio de tiempo de espera clasificadora: {self.tiempo_espera_clasificadora/60}")


    def schedule_event(self, tipo, t, clinica):
        # Agregar evento a la lista
        event = Event(tipo, t, clinica)
        self.upcoming_events.append(event)
        # Ordenar la lista de eventos por tiempo para ejecutar el evento más próximo primero
        self.upcoming_events.sort(key=lambda e: e.t)

    def step(self):
        # Avanza un paso en la simulación
        self.schedule.step()

        # Manejar eventos
        while self.upcoming_events:
            # Tomar el próximo evento
            event = self.upcoming_events.pop(0)
            
            # Si el evento ocurre después del tiempo de finalización, terminamos la simulación
            if event.t > 1440:
                break

            # Avanzar el tiempo
            self.current_time = event.t

            # Ejecutar el evento
            self.execute_event(event)

    def clinica_disponible(self, tipo_clinica=None):
        for clinica in self.clinicas:
            if isinstance(clinica, tipo_clinica) and clinica.esta_disponible():
                return clinica
        return None
    
    def execute_event(self, event):
        if event.tipo == "liberar_clinica":
            event.clinica.liberar()


### 3. Ejecución:

Se crear una instancia del modelo y se ejecuta:

In [84]:
model = HospitalModel(400)  

for i in range(1440):  
    model.step()

model.reporte()


Cantidad de pacientes que entraron: 400
Cantidad de pacientes de urgencias: 166
Cantidad de pacientes de enfermedad común: 217
Cantidad de pacientes de clasificación 1: 10
Promedio de tiempo de espera enfermedad común: 39.54838709677419
Promedio de tiempo de espera urgencias: 42.265060240963855
Promedio de tiempo de espera clasificación 1: 44.6
Promedio de tiempo de espera clasificadora: 259.96666666666664
